# **Set Environment**

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Oct 10 04:52:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import re
import pandas as pd

In [ ]:
!pip install transformers==3.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.5.1-py3-none-any.whl (1.3 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 3.5.0
    Uninstalling transformers-3.5.0:
      Successfully uninstalled transformers-3.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thai2transformers 0.1.2 requires torch>=1.5.0, but you have torch 1.4.0 which is incompatible.
thai2transformers 0.1.2 requires transformers==3.5.0, but you have transformers 3.5.1 which is incompatible.


In [ ]:
!pip install thai2transformers==0.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl (776.3 MB)
  Using cached transformers-3.5.0-py3-none-any.whl (1.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
  Attempting uninstall: transformers
    Found existing installation: transformers 3.5.1
    Uninstalling transformers-3.5.1:
      Successfully uninstalled transformers-3.5.1


In [ ]:
!pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.4.0 which is incompatible.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.4.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.4.0 which is incompatible.
thai2transformers 0.1.2 requires torch>=1.5.0, but you have torch 1.4.0 which is incompatible.
pytorch-lightning 1.7.7 requires torch>=1.9.*, but you have torch 1.4.0 which is incompatible.
fastai 2.7.9 requires torch<1.14,>=1.7, but you have torch 1.4.0 which is incompatible.


In [ ]:
pd.set_option('display.max_colwidth',1000)

# **Dataset**

Sentence level --- Intra-Sentential

1.   Relative clauses
2.   Topicalization
3. Serial Verb - ไม่ทำเพราะว่าถ้า mask แล้วก็จะไม่ธรรมชาติ
4. Embedded & Conjunction (Coordinate conjunction / Subordinating conjunction / Correlative conjunction)

Discourse level --- Inter-Sentential & Exophoric
- ระดับคนละประโยคกัน หรือไม่ก็ reference อยู่นอกบริบท
- ก่อนหน้านี้จะใช้ centering theory กัน

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/Colab_Notebooks/IS/Test_IS

/content/drive/MyDrive/Colab_Notebooks/IS/Test_IS


In [ ]:
train_df = pd.read_csv('train_df.csv', encoding='utf-8-sig')
dev_df = pd.read_csv('dev_df.csv', encoding='utf-8-sig')
test_df = pd.read_csv('test_df.csv', encoding='utf-8-sig')

In [ ]:
df = pd.concat([train_df,dev_df,test_df]).reset_index(drop=True)

In [ ]:
df.shape

(300, 7)

In [ ]:
df['MASK_NUM'].value_counts()

<mask_3rd>    116
<mask_2nd>     93
<mask_1st>     91
Name: MASK_NUM, dtype: int64

# **Functions**

In [ ]:
def clean_data(txt):
  #txt = txt.("")
  tx = re.sub("[0-9]+:[0-9]+\s(\w)+\*", "", txt)
  tx = re.sub("[0-9]+:[0-9]+\s(\w)+\"", "", tx)
  tx = re.sub("[0-9]+:[0-9]+\s(\w)+", "", tx)
  return tx

In [ ]:
def make_str(txt):
  return str(txt)

In [ ]:
#Max scores
def get_max_score(candidate_masks):
    sorted_lst = sorted(candidate_masks, key=lambda x: x['score']) #key => sort ตามอะไร
    #sorted_str = sorted_lst[-1]['sequence']
    sorted_score = sorted_lst[-1]['score']
    sorted_word = sorted_lst[-1]['token_str']
    return sorted_word + ' ' + '---' + ' ' + str(sorted_score)

In [ ]:
#Second
def get_second_score(candidate_masks):
    sorted_lst = sorted(candidate_masks, key=lambda x: x['score']) #key => sort ตามอะไร
    #sorted_str = sorted_lst[-2]['sequence']
    sorted_score = sorted_lst[-2]['score']
    sorted_word = sorted_lst[-2]['token_str']
    return sorted_word + ' ' + '---' + ' ' + str(sorted_score)

In [ ]:
#Third
def get_third_score(candidate_masks):
    sorted_lst = sorted(candidate_masks, key=lambda x: x['score']) #key => sort ตามอะไร
    #sorted_str = sorted_lst[-3]['sequence']
    sorted_score = sorted_lst[-3]['score']
    sorted_word = sorted_lst[-3]['token_str']
    return sorted_word + ' ' + '---' + ' ' + str(sorted_score)

In [ ]:
#fourth
def get_fourth_score(candidate_masks):
    sorted_lst = sorted(candidate_masks, key=lambda x: x['score']) #key => sort ตามอะไร
    #sorted_str = sorted_lst[-4]['sequence']
    sorted_score = sorted_lst[-4]['score']
    sorted_word = sorted_lst[-4]['token_str']
    return sorted_word + ' ' + '---' + ' ' + str(sorted_score)

In [ ]:
#fifth
def get_fifth_score(candidate_masks):
    sorted_lst = sorted(candidate_masks, key=lambda x: x['score']) #key => sort ตามอะไร
    #sorted_str = sorted_lst[-5]['sequence']
    sorted_score = sorted_lst[-5]['score']
    sorted_word = sorted_lst[-5]['token_str']
    return sorted_word + ' ' + '---' + ' ' + str(sorted_score)

In [ ]:
#pandas ทำตารางเรียง score
#[[{'score': 0.019775109365582466, 'sequence': '<s> ผู้ชายที่กําลังพูดโทรศัพท์<_> ไม่ค่อยชอบฉัน</s>', 'token': 8, 'token_str': '<_>'},
#str(fillmask_intra[0][3]['score']) + ' + ' + str(fillmask_intra[0][3]['token_str']

def show_table(dict_fillmask):
  df = pd.DataFrame({"ranking" : [], "score" : [], "sequence" : [], "token_str" : []})
  
  #list
  for l in dict_fillmask: 
    i = 0
    
    #dict
    new_row = {}
    for d in l:
      i += 1
      i_int = int(round(i))
      new_row['ranking'] = str(i_int)
      new_row['score'] = d['score']
      new_row['sequence'] = d['sequence']
      new_row['token_str'] = d['token_str']
    
      df = df.append(new_row, ignore_index=True)

  return df

# **MLM**

In [ ]:
from transformers import (
    CamembertTokenizer,
    AutoModelForMaskedLM,
    pipeline)
from thai2transformers.preprocess import process_transformers

In [ ]:
#thai2transformers

import thai2transformers
from thai2transformers.preprocess import process_transformers
from thai2transformers.metrics import (
    classification_metrics, 
    multilabel_classification_metrics,
)
from thai2transformers.tokenizers import (
    ThaiRobertaTokenizer,
    ThaiWordsNewmmTokenizer,
    ThaiWordsSyllableTokenizer,
    FakeSefrCutTokenizer,
    SEFR_SPLIT_TOKEN
)

In [ ]:
# Load pre-trained tokenizer
tokenizer = CamembertTokenizer.from_pretrained(
                                  'airesearch/wangchanberta-base-att-spm-uncased',
                                  revision='main')
tokenizer.additional_special_tokens = ['<s>NOTUSED', '</s>NOTUSED', '<_>']

In [ ]:
# Load pre-trained model
model = AutoModelForMaskedLM.from_pretrained(
                                  'airesearch/wangchanberta-base-att-spm-uncased',
                                  revision='main')

In [ ]:
# Pipeline
fill_mask = pipeline(task='fill-mask',
          tokenizer=tokenizer,
          model=model)

In [ ]:
def fill_mask_pad(input_text):
    return fill_mask(input_text+'<pad>')

# MLM to data

In [ ]:
pattern = r"mask"
df['text'] = df['text'].apply(lambda x: re.sub(pattern, "<mask>",x))

In [ ]:
df.head(5)

,text,MASK_NUM,PRON,REF,tokens,num_label,n_tokens
0,ปีเตอร์นะ ฉันได้ยินข่าวว่าจอห์นเพิ่งพา<mask>ไปโรงพยาบาลเมื่อเช้านี้,<mask_3rd>,เขา,ปีเตอร์,ปีเตอร์|นะ| |ฉัน|ได้ยิน|ข่าว|ว่า|จอห์น|เพิ่ง|พา|mask|ไป|โรงพยาบาล|เมื่อเช้านี้,N|N|N|N|N|N|N|N|N|N|mask_3rd|N|N|N,14
1,ทำไมส่วนใหญ่ญาติต้องอิจฉากันเอง ทำไม<mask>ถึงไม่รักกัน,<mask_3rd>,พวกเขา,ญาติ,ทำไม|ส่วนใหญ่|ญาติ|ต้อง|อิจฉา|กันเอง| |ทำไม|mask|ถึง|ไม่|รัก|กัน,N|N|N|N|N|N|N|N|mask_3rd|N|N|N|N,13
2,เมื่อก่อนตอนเธออ้วน <mask>ดูแก่กว่าปัจจุบันอีก,<mask_2nd>,เธอ,เธอ,เมื่อก่อน|ตอน|เธอ|อ้วน| |mask|ดู|แก่|กว่า|ปัจจุบัน|อีก,N|N|N|N|N|mask_2nd|N|N|N|N|N,11
3,ไปถามพี่เขาหน่อยซิว่า<mask>อยากจะกินข้าวที่ไหน,<mask_3rd>,เขา,พี่,ไป|ถาม|พี่|เขา|หน่อย|ซิ|ว่า|mask|อยาก|จะ|กินข้าว|ที่ไหน,N|N|N|N|N|N|N|mask_3rd|N|N|N|N,12
4,เฮียแกกลับมาร้องเพลงได้แล้ว สงสัย<mask>เลิกยาถาวรแล้ว,<mask_3rd>,เขา,เฮีย,เฮีย|แก|กลับมา|ร้องเพลง|ได้|แล้ว| |สงสัย|mask|เลิก|ยา|ถาวร|แล้ว,N|N|N|N|N|N|N|N|mask_3rd|N|N|N|N,13


In [ ]:
df['processed_input_intra'] = df['text'].apply(make_str)
df['processed_input_intra'] = df['text'].apply(process_transformers)

In [ ]:
df['fill_mask'] = df['processed_input_intra'].apply(fill_mask)

# Get scores

In [ ]:
df['1st_candidate'] = df['fill_mask'].apply(get_max_score)
df['2nd_candidate'] = df['fill_mask'].apply(get_second_score)
df['3rd_candidate'] = df['fill_mask'].apply(get_third_score)
df['4th_candidate'] = df['fill_mask'].apply(get_fourth_score)
df['5th_candidate'] = df['fill_mask'].apply(get_fifth_score)

In [ ]:
df['text'] = df['text'].apply(lambda x: re.sub('<mask>', '_____' , x))

In [ ]:
df.head(2)

,text,MASK_NUM,PRON,REF,tokens,num_label,n_tokens,processed_input_intra,fill_mask,1st_candidate,2nd_candidate,3rd_candidate,4th_candidate,5th_candidate
0,ปีเตอร์นะ ฉันได้ยินข่าวว่าจอห์นเพิ่งพา_____ไปโรงพยาบาลเมื่อเช้านี้,<mask_3rd>,เขา,ปีเตอร์,ปีเตอร์|นะ| |ฉัน|ได้ยิน|ข่าว|ว่า|จอห์น|เพิ่ง|พา|mask|ไป|โรงพยาบาล|เมื่อเช้านี้,N|N|N|N|N|N|N|N|N|N|mask_3rd|N|N|N,14,ปีเตอร์นะ<_>ฉันได้ยินข่าวว่าจอห์นเพิ่งพา<mask>ไปโรงพยาบาลเมื่อเช้านี้,"[{'sequence': '<s> ปีเตอร์นะ<_> ฉันได้ยินข่าวว่าจอห์นเพิ่งพาปีเตอร์ ไปโรงพยาบาลเมื่อเช้านี้</s>', 'score': 0.3155905604362488, 'token': 7171, 'token_str': 'ปีเตอร์'}, {'sequence': '<s> ปีเตอร์นะ<_> ฉันได้ยินข่าวว่าจอห์นเพิ่งพาจอห์น ไปโรงพยาบาลเมื่อเช้านี้</s>', 'score': 0.08104077726602554, 'token': 3425, 'token_str': 'จอห์น'}, {'sequence': '<s> ปีเตอร์นะ<_> ฉันได้ยินข่าวว่าจอห์นเพิ่งพาคุณ ไปโรงพยาบาลเมื่อเช้านี้</s>', 'score': 0.049786604940891266, 'token': 77, 'token_str': 'คุณ'}, {'sequence': '<s> ปีเตอร์นะ<_> ฉันได้ยินข่าวว่าจอห์นเพิ่งพาภรรยา ไปโรงพยาบาลเมื่อเช้านี้</s>', 'score': 0.049653030931949615, 'token': 1520, 'token_str': 'ภรรยา'}, {'sequence': '<s> ปีเตอร์นะ<_> ฉันได้ยินข่าวว่าจอห์นเพิ่งพาครอบครัว ไปโรงพยาบาลเมื่อเช้านี้</s>', 'score': 0.01633046381175518, 'token': 359, 'token_str': 'ครอบครัว'}]",ปีเตอร์ --- 0.3155905604362488,จอห์น --- 0.08104077726602554,คุณ --- 0.049786604940891266,ภรรยา --- 0.049653030931949615,ครอบครัว --- 0.01633046381175518
1,ทำไมส่วนใหญ่ญาติต้องอิจฉากันเอง ทำไม_____ถึงไม่รักกัน,<mask_3rd>,พวกเขา,ญาติ,ทำไม|ส่วนใหญ่|ญาติ|ต้อง|อิจฉา|กันเอง| |ทำไม|mask|ถึง|ไม่|รัก|กัน,N|N|N|N|N|N|N|N|mask_3rd|N|N|N|N,13,ทำไมส่วนใหญ่ญาติต้องอิจฉากันเอง<_>ทำไม<mask>ถึงไม่รักกัน,"[{'sequence': '<s> ทําไมส่วนใหญ่ญาติต้องอิจฉากันเอง<_> ทําไมญาติ ถึงไม่รักกัน</s>', 'score': 0.17374372482299805, 'token': 1160, 'token_str': 'ญาติ'}, {'sequence': '<s> ทําไมส่วนใหญ่ญาติต้องอิจฉากันเอง<_> ทําไมพี่น้อง ถึงไม่รักกัน</s>', 'score': 0.1270301192998886, 'token': 1067, 'token_str': 'พี่น้อง'}, {'sequence': '<s> ทําไมส่วนใหญ่ญาติต้องอิจฉากันเอง<_> ทําไมคนไทย ถึงไม่รักกัน</s>', 'score': 0.08553747832775116, 'token': 828, 'token_str': 'คนไทย'}, {'sequence': '<s> ทําไมส่วนใหญ่ญาติต้องอิจฉากันเอง<_> ทําไมลูกหลาน ถึงไม่รักกัน</s>', 'score': 0.06577395647764206, 'token': 3915, 'token_str': 'ลูกหลาน'}, {'sequence': '<s> ทําไมส่วนใหญ่ญาติต้องอิจฉากันเอง<_> ทําไมมนุษย์ ถึงไม่รักกัน</s>', 'score': 0.055211346596479416, 'token': 778, 'token_str': 'มนุษย์'}]",ญาติ --- 0.17374372482299805,พี่น้อง --- 0.1270301192998886,คนไทย --- 0.08553747832775116,ลูกหลาน --- 0.06577395647764206,มนุษย์ --- 0.055211346596479416


# Result - no padding

In [ ]:
df[['text','PRON', 'MASK_NUM', '1st_candidate', '2nd_candidate','3rd_candidate', '4th_candidate', '5th_candidate']].sort_values(by='MASK_NUM')

,text,PRON,MASK_NUM,1st_candidate,2nd_candidate,3rd_candidate,4th_candidate,5th_candidate
107,เพราะว่ากูเมื่อยไง _____ก็เลยต้องไปนวด,กู,<mask_1st>,กู --- 0.12162774056196213,อารมณ์ --- 0.09134655445814133,อาจารย์ --- 0.08278273046016693,5 --- 0.04895583167672157,สงกรานต์ --- 0.034639712423086166
247,งั้นพวกฉันขอลงไปกินขนมก่อนนะ เดี๋ยว_____จะเก็บขนมไว้ให้พวกเธอนะจ๊ะ,พวกฉัน,<mask_1st>,▁พรุ่งนี้ --- 0.33088964223861694,พรุ่งนี้ --- 0.2205573320388794,▁ฉัน --- 0.026100382208824158,ตอนเย็น --- 0.020557917654514313,▁เดี๋ยว --- 0.020137157291173935
134,ทุกวันนี้_____ก็ยังไม่เข้าใจว่าทำไมต้องเสียเวลาวันละ 3 ชั่วโมงกับการเข้าออฟฟิศทั้งๆ ที่เนื้องานของเราทำงานจากที่ไหนก็ได้ 100%,เรา,<mask_1st>,<_> --- 0.7869062423706055,เรา --- 0.01308024674654007,ผม --- 0.010976910591125488,... --- 0.010136368684470654,ดิฉัน --- 0.008493795059621334
132,เมื่อเช้า_____เข้าไปคุยกับอาจารย์สมหมายมา เราถามอาจารย์ให้แล้วนะเรื่องวิชาที่เปิดใหม่,เรา,<mask_1st>,เพิ่ง --- 0.19559547305107117,เรา --- 0.08714921027421951,▁เรา --- 0.08162175118923187,พึ่ง --- 0.059208162128925323,ได้มีโอกาส --- 0.05542902275919914
128,พรุ่งนี้น้องเอาหนังสือมาให้พี่ เดี๋ยว_____เอาไปให้อาจารย์ให้เอง,พี่,<mask_1st>,▁พี่ --- 0.25188371539115906,พี่ --- 0.16415007412433624,เดี๋ยว --- 0.13060976564884186,▁พรุ่งนี้ --- 0.10224680602550507,▁เดี๋ยว --- 0.09391709417104721
...,...,...,...,...,...,...,...,...
148,ฉันโทรไปชวนนางแล้ว แต่_____ไม่ยอมมา,นาง,<mask_3rd>,นาง --- 0.5641579031944275,▁นาง --- 0.05752906575798988,ปฏิเสธ --- 0.015258782543241978,เพื่อน --- 0.014806126244366169,โทรศัพท์ --- 0.012289668433368206
150,ตอน_____ไปปีนเขา หลานก้อถามหาป้านุ๊ก,เขา,<mask_3rd>,หลาน --- 0.04460876062512398,ปิดเทอม --- 0.033313725143671036,สงกรานต์ --- 0.03134149685502052,ยาย --- 0.030819358304142952,ป้า --- 0.028578316792845726
152,เด็กคนนั้นน่ะ ครูบอกว่า_____ได้รางวัล,เขา,<mask_3rd>,ฉัน --- 0.05227363109588623,หนู --- 0.04616459086537361,คุณ --- 0.04170309007167816,เธอ --- 0.039369724690914154,▁หนู --- 0.03528561070561409
159,รางวัลนั่นน่ะ เด็กคนนั้นได้รับ_____ไป,มัน,<mask_3rd>,รางวัล --- 0.13164420425891876,สิทธิ์ --- 0.07681873440742493,เกียรติ --- 0.043138690292835236,มงกุฎ --- 0.04172616824507713,ของขวัญ --- 0.029111284762620926


In [ ]:
df.loc[[54, 62, 272, 37, 14, 244, 180, 15, 56, 215]][['text','PRON', 'MASK_NUM', '1st_candidate', '2nd_candidate','3rd_candidate', '4th_candidate', '5th_candidate']]

,text,PRON,MASK_NUM,1st_candidate,2nd_candidate,3rd_candidate,4th_candidate,5th_candidate
54,แมรี่ไม่เคยรักพอลเลย _____เป็นแค่คนคั่นเวลาเติมเต็มที่ว่างในใจของเธอเท่านั้น,เขา,<mask_3rd>,แมรี่ --- 0.2776719033718109,พอล --- 0.07014447450637817,ปีเตอร์ --- 0.06722275167703629,ชาร์ล --- 0.04526475816965103,จอห์น --- 0.04023551195859909
62,นางเอกโดนคนในบ้านของพระเอกกลั่นแกล้งสารพัด แต่_____ก็ยังอดทนและผ่านมันไปได้,เธอ,<mask_3rd>,พระเอก --- 0.5712325572967529,นางเอก --- 0.08089285343885422,ตัวเอง --- 0.0322355255484581,อาจารย์ --- 0.013593444600701332,ปีเตอร์ --- 0.010575656779110432
272,เต้ยอายุเท่าไหร่วะ กูเห็นบอมเรียก_____ว่าน้อง กูงงเลย,เขา,<mask_3rd>,เต้ย --- 0.14747962355613708,ไอซ์ --- 0.08217374980449677,เจมส์ --- 0.05392340198159218,อเล็กซ์ --- 0.03732038661837578,เฟิร์น --- 0.028343481943011284
37,เราชอบฟังเพลงนี้อ่ะ เธอเคยฟัง_____ป่ะ,มัน,<mask_3rd>,อยู่ --- 0.10478004068136215,ฉัน --- 0.07512647658586502,เหมือนกัน --- 0.06734223663806915,ต้นฉบับ --- 0.04743954911828041,เพลงนี้ --- 0.0393141433596611
14,กูรับเค้กมาแล้วนะ เดี๋ยวกูเอา_____ไปฝากร้านไว้,มัน,<mask_3rd>,ของขวัญ --- 0.05157982185482979,เค้ก --- 0.04810033738613129,ช็อกโกแลต --- 0.0466129295527935,โทรศัพท์ --- 0.04011963680386543,ตุ๊กตา --- 0.03868008777499199
244,อาจารย์สมเกียรติบอกว่าวันนี้_____เข้าประชุมไม่ได้นะ,เขา,<mask_3rd>,อาจารย์ --- 0.44005799293518066,▁อาจารย์ --- 0.07328034937381744,นักศึกษา --- 0.041657235473394394,จารย์ --- 0.029423033818602562,ประยุทธ์ --- 0.021674297749996185
180,พี่พิงค์คะ เปนงายมั่งอ่า มุกไม่เจอ_____นานเรย,พี่,<mask_2nd>,หน้า --- 0.2053729146718979,พี่ --- 0.1835322082042694,มา --- 0.04282321408390999,กัน --- 0.03376293554902077,<_> --- 0.029974214732646942
15,หมอคิดว่าพรุ่งนี้คนไข้น่าจะไม่ต้องมาหา_____แล้วนะครับ,ผม,<mask_1st>,จิตแพทย์ --- 0.18674401938915253,อาจารย์ --- 0.11315929889678955,แพทย์ --- 0.06022386997938156,ญาติ --- 0.053173743188381195,หมอ --- 0.051458295434713364
56,ป้าทำความสะอาดห้องให้เสร็จแล้วค่ะ เดี๋ยว_____เอากุญแจไปใส่ไว้ในช่องจดหมายของคุณไว้นะคะ,ป้า,<mask_1st>,คุณ --- 0.33849868178367615,กรุณา --- 0.05750815197825432,พรุ่งนี้ --- 0.0542270764708519,เดี๋ยว --- 0.045171696692705154,ป้า --- 0.031461868435144424
215,ลูกค้าทำไฟล์งานขนาดที่ต้องการตัดแบ่งมาให้ทางร้านได้เลยนะคะ เดี่ยว_____จะนำงานลูกค้ามาวางลง A3 ให้เองนะค้า,เรา,<mask_1st>,ทางร้าน --- 0.801447331905365,แอดมิน --- 0.07501468807458878,แม่ค้า --- 0.022449638694524765,ทีมงาน --- 0.013112553395330906,แอด --- 0.01010211557149887


In [ ]:
# data กำกวม
df.loc[[192, 11]][['text','PRON', 'MASK_NUM', '1st_candidate', '2nd_candidate','3rd_candidate', '4th_candidate', '5th_candidate']]

,text,PRON,MASK_NUM,1st_candidate,2nd_candidate,3rd_candidate,4th_candidate,5th_candidate
192,บ้านผมมีคนติดโควิดครับ แต่เมื่อเช้า_____ตรวจเอทีเคแล้ว ยังไม่เป็นอะไร,ผม,<mask_1st>,ศูนย์ --- 0.06224599480628967,โรงพยาบาล --- 0.060972001403570175,แพทย์ --- 0.04747272655367851,ตํารวจ --- 0.03496158868074417,สาธารณสุข --- 0.02730073407292366
11,พ่อขับรถออกไปแล้วค่ะ เพราะ_____รีบไปทำงาน,เขา,<mask_3rd>,หนู --- 0.09610312432050705,▁พ่อ --- 0.08090634644031525,▁หนู --- 0.04850921034812927,▁ฉัน --- 0.04774780943989754,▁แม่ --- 0.04657074809074402


# Test fill mask pad

In [ ]:
df_pad = df

In [ ]:
df_pad['fill_mask'] = df_pad['processed_input_intra'].apply(fill_mask_pad)

In [ ]:
df_pad.head(2)

,text,MASK_NUM,PRON,REF,tokens,num_label,n_tokens,processed_input_intra,fill_mask,1st_candidate,2nd_candidate,3rd_candidate,4th_candidate,5th_candidate
0,ปีเตอร์นะ ฉันได้ยินข่าวว่าจอห์นเพิ่งพา_____ไปโรงพยาบาลเมื่อเช้านี้,<mask_3rd>,เขา,ปีเตอร์,ปีเตอร์|นะ| |ฉัน|ได้ยิน|ข่าว|ว่า|จอห์น|เพิ่ง|พา|mask|ไป|โรงพยาบาล|เมื่อเช้านี้,N|N|N|N|N|N|N|N|N|N|mask_3rd|N|N|N,14,ปีเตอร์นะ<_>ฉันได้ยินข่าวว่าจอห์นเพิ่งพา<mask>ไปโรงพยาบาลเมื่อเช้านี้,"[{'sequence': '<s> ปีเตอร์นะ<_> ฉันได้ยินข่าวว่าจอห์นเพิ่งพาปีเตอร์ ไปโรงพยาบาลเมื่อเช้านี้</s>', 'score': 0.43101000785827637, 'token': 7171, 'token_str': 'ปีเตอร์'}, {'sequence': '<s> ปีเตอร์นะ<_> ฉันได้ยินข่าวว่าจอห์นเพิ่งพาจอห์น ไปโรงพยาบาลเมื่อเช้านี้</s>', 'score': 0.066167913377285, 'token': 3425, 'token_str': 'จอห์น'}, {'sequence': '<s> ปีเตอร์นะ<_> ฉันได้ยินข่าวว่าจอห์นเพิ่งพาคุณ ไปโรงพยาบาลเมื่อเช้านี้</s>', 'score': 0.0596366822719574, 'token': 77, 'token_str': 'คุณ'}, {'sequence': '<s> ปีเตอร์นะ<_> ฉันได้ยินข่าวว่าจอห์นเพิ่งพาภรรยา ไปโรงพยาบาลเมื่อเช้านี้</s>', 'score': 0.029679423198103905, 'token': 1520, 'token_str': 'ภรรยา'}, {'sequence': '<s> ปีเตอร์นะ<_> ฉันได้ยินข่าวว่าจอห์นเพิ่งพาฉัน ไปโรงพยาบาลเมื่อเช้านี้</s>', 'score': 0.018187865614891052, 'token': 106, 'token_str': 'ฉัน'}]",ปีเตอร์ --- 0.43101000785827637,จอห์น --- 0.066167913377285,คุณ --- 0.0596366822719574,ภรรยา --- 0.029679423198103905,ฉัน --- 0.018187865614891052
1,ทำไมส่วนใหญ่ญาติต้องอิจฉากันเอง ทำไม_____ถึงไม่รักกัน,<mask_3rd>,พวกเขา,ญาติ,ทำไม|ส่วนใหญ่|ญาติ|ต้อง|อิจฉา|กันเอง| |ทำไม|mask|ถึง|ไม่|รัก|กัน,N|N|N|N|N|N|N|N|mask_3rd|N|N|N|N,13,ทำไมส่วนใหญ่ญาติต้องอิจฉากันเอง<_>ทำไม<mask>ถึงไม่รักกัน,"[{'sequence': '<s> ทําไมส่วนใหญ่ญาติต้องอิจฉากันเอง<_> ทําไมญาติ ถึงไม่รักกัน</s>', 'score': 0.1679929941892624, 'token': 1160, 'token_str': 'ญาติ'}, {'sequence': '<s> ทําไมส่วนใหญ่ญาติต้องอิจฉากันเอง<_> ทําไมลูกหลาน ถึงไม่รักกัน</s>', 'score': 0.08211040496826172, 'token': 3915, 'token_str': 'ลูกหลาน'}, {'sequence': '<s> ทําไมส่วนใหญ่ญาติต้องอิจฉากันเอง<_> ทําไมพี่น้อง ถึงไม่รักกัน</s>', 'score': 0.0738968625664711, 'token': 1067, 'token_str': 'พี่น้อง'}, {'sequence': '<s> ทําไมส่วนใหญ่ญาติต้องอิจฉากันเอง<_> ทําไมมนุษย์ ถึงไม่รักกัน</s>', 'score': 0.07272123545408249, 'token': 778, 'token_str': 'มนุษย์'}, {'sequence': '<s> ทําไมส่วนใหญ่ญาติต้องอิจฉากันเอง<_> ทําไมคนไทย ถึงไม่รักกัน</s>', 'score': 0.07164447754621506, 'token': 828, 'token_str': 'คนไทย'}]",ญาติ --- 0.1679929941892624,ลูกหลาน --- 0.08211040496826172,พี่น้อง --- 0.0738968625664711,มนุษย์ --- 0.07272123545408249,คนไทย --- 0.07164447754621506


In [ ]:
df_pad['1st_candidate'] = df_pad['fill_mask'].apply(get_max_score)
df_pad['2nd_candidate'] = df_pad['fill_mask'].apply(get_second_score)
df_pad['3rd_candidate'] = df_pad['fill_mask'].apply(get_third_score)
df_pad['4th_candidate'] = df_pad['fill_mask'].apply(get_fourth_score)
df_pad['5th_candidate'] = df_pad['fill_mask'].apply(get_fifth_score)

In [ ]:
df_pad['text'] = df_pad['text'].apply(lambda x: re.sub('<mask>', '_____' , x))

# Result - with padding

In [ ]:
df_pad[['text','PRON', 'MASK_NUM', '1st_candidate', '2nd_candidate','3rd_candidate', '4th_candidate', '5th_candidate']].sort_values(by='MASK_NUM')

,text,PRON,MASK_NUM,1st_candidate,2nd_candidate,3rd_candidate,4th_candidate,5th_candidate
107,เพราะว่ากูเมื่อยไง _____ก็เลยต้องไปนวด,กู,<mask_1st>,กู --- 0.12162774056196213,อารมณ์ --- 0.09134655445814133,อาจารย์ --- 0.08278273046016693,5 --- 0.04895583167672157,สงกรานต์ --- 0.034639712423086166
247,งั้นพวกฉันขอลงไปกินขนมก่อนนะ เดี๋ยว_____จะเก็บขนมไว้ให้พวกเธอนะจ๊ะ,พวกฉัน,<mask_1st>,▁พรุ่งนี้ --- 0.33088964223861694,พรุ่งนี้ --- 0.2205573320388794,▁ฉัน --- 0.026100382208824158,ตอนเย็น --- 0.020557917654514313,▁เดี๋ยว --- 0.020137157291173935
134,ทุกวันนี้_____ก็ยังไม่เข้าใจว่าทำไมต้องเสียเวลาวันละ 3 ชั่วโมงกับการเข้าออฟฟิศทั้งๆ ที่เนื้องานของเราทำงานจากที่ไหนก็ได้ 100%,เรา,<mask_1st>,<_> --- 0.7869062423706055,เรา --- 0.01308024674654007,ผม --- 0.010976910591125488,... --- 0.010136368684470654,ดิฉัน --- 0.008493795059621334
132,เมื่อเช้า_____เข้าไปคุยกับอาจารย์สมหมายมา เราถามอาจารย์ให้แล้วนะเรื่องวิชาที่เปิดใหม่,เรา,<mask_1st>,เพิ่ง --- 0.19559547305107117,เรา --- 0.08714921027421951,▁เรา --- 0.08162175118923187,พึ่ง --- 0.059208162128925323,ได้มีโอกาส --- 0.05542902275919914
128,พรุ่งนี้น้องเอาหนังสือมาให้พี่ เดี๋ยว_____เอาไปให้อาจารย์ให้เอง,พี่,<mask_1st>,▁พี่ --- 0.25188371539115906,พี่ --- 0.16415007412433624,เดี๋ยว --- 0.13060976564884186,▁พรุ่งนี้ --- 0.10224680602550507,▁เดี๋ยว --- 0.09391709417104721
...,...,...,...,...,...,...,...,...
148,ฉันโทรไปชวนนางแล้ว แต่_____ไม่ยอมมา,นาง,<mask_3rd>,นาง --- 0.5641579031944275,▁นาง --- 0.05752906575798988,ปฏิเสธ --- 0.015258782543241978,เพื่อน --- 0.014806126244366169,โทรศัพท์ --- 0.012289668433368206
150,ตอน_____ไปปีนเขา หลานก้อถามหาป้านุ๊ก,เขา,<mask_3rd>,หลาน --- 0.04460876062512398,ปิดเทอม --- 0.033313725143671036,สงกรานต์ --- 0.03134149685502052,ยาย --- 0.030819358304142952,ป้า --- 0.028578316792845726
152,เด็กคนนั้นน่ะ ครูบอกว่า_____ได้รางวัล,เขา,<mask_3rd>,ฉัน --- 0.05227363109588623,หนู --- 0.04616459086537361,คุณ --- 0.04170309007167816,เธอ --- 0.039369724690914154,▁หนู --- 0.03528561070561409
159,รางวัลนั่นน่ะ เด็กคนนั้นได้รับ_____ไป,มัน,<mask_3rd>,รางวัล --- 0.13164420425891876,สิทธิ์ --- 0.07681873440742493,เกียรติ --- 0.043138690292835236,มงกุฎ --- 0.04172616824507713,ของขวัญ --- 0.029111284762620926


In [ ]:
df.iloc[[103]]

,text,MASK_NUM,PRON,REF,tokens,num_label,n_tokens,processed_input_intra,fill_mask,1st_candidate,2nd_candidate,3rd_candidate,4th_candidate,5th_candidate
103,เวลา_____พูดอย่าละประธานสิครับ คนอื่นเขาเข้าใจผิดหมด,<mask_2nd>,คุณ,คุณ,เวลา|mask|พูด|อย่า|ละ|ประธาน|สิ|ครับ| |คนอื่น|เขา|เข้าใจผิด|หมด,N|mask_2nd|N|N|N|N|N|N|N|N|N|N|N,13,เวลา<mask>พูดอย่าละประธานสิครับ<_>คนอื่นเขาเข้าใจผิดหมด,"[{'sequence': '<s> เวลาหาเสียง พูดอย่าละประธานสิครับ<_> คนอื่นเขาเข้าใจผิดหมด</s>', 'score': 0.13038350641727448, 'token': 4141, 'token_str': 'หาเสียง'}, {'sequence': '<s> เวลาคุณ พูดอย่าละประธานสิครับ<_> คนอื่นเขาเข้าใจผิดหมด</s>', 'score': 0.11775675415992737, 'token': 77, 'token_str': 'คุณ'}, {'sequence': '<s> เวลาอภิปราย พูดอย่าละประธานสิครับ<_> คนอื่นเขาเข้าใจผิดหมด</s>', 'score': 0.11068163812160492, 'token': 5409, 'token_str': 'อภิปราย'}, {'sequence': '<s> เวลาสัมภาษณ์ พูดอย่าละประธานสิครับ<_> คนอื่นเขาเข้าใจผิดหมด</s>', 'score': 0.10788112133741379, 'token': 1271, 'token_str': 'สัมภาษณ์'}, {'sequence': '<s> เวลาพูด พูดอย่าละประธานสิครับ<_> คนอื่นเขาเข้าใจผิดหมด</s>', 'score': 0.10541140288114548, 'token': 246, 'token_str': 'พูด'}]",หาเสียง --- 0.13038350641727448,คุณ --- 0.11775675415992737,อภิปราย --- 0.11068163812160492,สัมภาษณ์ --- 0.10788112133741379,พูด --- 0.10541140288114548


In [ ]:
df_pad.iloc[[103]]

,text,MASK_NUM,PRON,REF,tokens,num_label,n_tokens,processed_input_intra,fill_mask,1st_candidate,2nd_candidate,3rd_candidate,4th_candidate,5th_candidate
103,เวลา_____พูดอย่าละประธานสิครับ คนอื่นเขาเข้าใจผิดหมด,<mask_2nd>,คุณ,คุณ,เวลา|mask|พูด|อย่า|ละ|ประธาน|สิ|ครับ| |คนอื่น|เขา|เข้าใจผิด|หมด,N|mask_2nd|N|N|N|N|N|N|N|N|N|N|N,13,เวลา<mask>พูดอย่าละประธานสิครับ<_>คนอื่นเขาเข้าใจผิดหมด,"[{'sequence': '<s> เวลาหาเสียง พูดอย่าละประธานสิครับ<_> คนอื่นเขาเข้าใจผิดหมด</s>', 'score': 0.13038350641727448, 'token': 4141, 'token_str': 'หาเสียง'}, {'sequence': '<s> เวลาคุณ พูดอย่าละประธานสิครับ<_> คนอื่นเขาเข้าใจผิดหมด</s>', 'score': 0.11775675415992737, 'token': 77, 'token_str': 'คุณ'}, {'sequence': '<s> เวลาอภิปราย พูดอย่าละประธานสิครับ<_> คนอื่นเขาเข้าใจผิดหมด</s>', 'score': 0.11068163812160492, 'token': 5409, 'token_str': 'อภิปราย'}, {'sequence': '<s> เวลาสัมภาษณ์ พูดอย่าละประธานสิครับ<_> คนอื่นเขาเข้าใจผิดหมด</s>', 'score': 0.10788112133741379, 'token': 1271, 'token_str': 'สัมภาษณ์'}, {'sequence': '<s> เวลาพูด พูดอย่าละประธานสิครับ<_> คนอื่นเขาเข้าใจผิดหมด</s>', 'score': 0.10541140288114548, 'token': 246, 'token_str': 'พูด'}]",หาเสียง --- 0.13038350641727448,คุณ --- 0.11775675415992737,อภิปราย --- 0.11068163812160492,สัมภาษณ์ --- 0.10788112133741379,พูด --- 0.10541140288114548
